<a href="https://colab.research.google.com/github/rahiakela/data-science-research-and-practice/blob/main/data-extraction-works/01-tabulapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reference**

https://www.geeksforgeeks.org/drop-rows-from-the-dataframe-based-on-certain-condition-applied-on-a-column/

https://datatofish.com/export-pandas-dataframe-json/

In [1]:
!pip install -q tabula-py

     |████████████████████████████████| 12.0 MB 8.2 MB/s 


In [2]:
import tabula
import pandas as pd

##Loading pdf into Pandas

In [16]:
# Read pdf into DataFrame
dfs = tabula.read_pdf("redacted1.pdf", lattice=True, relative_area=True, multiple_tables=True, pages="all")

# convert PDF into CSV
#tabula.convert_into("redacted1.pdf", "output.csv", output_format="csv")

Oct 24, 2022 6:26:53 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
len(dfs)

5

In [ ]:
dfs[1].columns[0]

'I951'

In [ ]:
dfs[2]["Test Name"][21]

In [ ]:
dfs[13]

In [5]:
def drop_unwanted_columns(df):
  wanted_columns = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']
  drop_columns = []
  for col in df.columns:
    if col in wanted_columns:
      drop_columns.append(col)
  data_df = df[df.columns.drop(drop_columns)]
  return data_df

In [6]:
def rename_columns(df):
  cols_list = ["Encounter", "Specimen", "Service Date", "High Score", "Score", "Lab", "Doctor Name", "Location", "Phone"]
  current_dict = {}
  for idx, col_name in enumerate(df.columns):
    current_dict[col_name] = cols_list[idx]
    # renaming the columns
    df.rename(columns=current_dict, inplace = True)
  return df

In [ ]:
def add_new_columns(df):
  cols_list = ["Encounter", "Specimen", "Service Date", "High Score", "Score", "Lab", "Doctor Name", "Location", "Phone"]
  test_column_list = ["Test Name", "Results","Reference Range", "Score", "Lab"]
  diagnosis_column_list = ["Diagnosis Code", "Description","Code Type", "Score"]
  doctor_sp_column_list = ["Doctor Specialty", "Score"]
  lab_column_list = ["Lab", "Name","Location", "Director", "CLIA"]
  current_dict = {}

  current_column = df.columns[0]
  # if current_column == "Test Name":
  for idx, col_name in enumerate(df.columns):
    #print(f"{cols_list[idx]} >> {col_name}")
    current_dict[cols_list[idx]] = col_name
    new_row_df = pd.DataFrame(current_dict, index =[0])  # adding a new row
    updated_df = pd.concat([new_row_df, df[:]], ignore_index=True) # .reset_index(drop = True)
  return new_row_df

In [44]:
def update_df_columns(df):
  col_list = ["Encounter", "Specimen", "Service Date", "High Score", "Score", "Lab", "Doctor Name", "Location", "Phone"]

  curr_rename_dict = {}
  curr_new_dict = {}

  current_column = df.columns[0]

  for idx, col_name in enumerate(df.columns):
    curr_rename_dict[col_name] = col_list[idx]

  for idx, col_name in enumerate(df.columns):
    curr_new_dict[col_list[idx]] = col_name

  # renaming the columns
  df.rename(columns=curr_rename_dict, inplace = True)

  # adding a new row
  new_row_df = pd.DataFrame(curr_new_dict, index =[0])
  # Concatenate new_row with df
  updated_df = pd.concat([new_row_df, df[:]]).reset_index(drop = True)
  return updated_df

In [48]:
my_list = ['9', '2018-03-20', '5', 'Score', 'Lab', 'Doctor Name', 'Location','Phone']
my_list

['9', '2018-03-20', '5', 'Score', 'Lab', 'Doctor Name', 'Location', 'Phone']

In [49]:
my_list.insert(1, "")
my_list

['9',
 '',
 '2018-03-20',
 '5',
 'Score',
 'Lab',
 'Doctor Name',
 'Location',
 'Phone']

In [54]:
d_list = list(data_df.columns)
d_list

['Test Name', 'Results', 'Reference Range', 'Score']

In [55]:
d_list.insert(1, "")
d_list

['Test Name', '', 'Results', 'Reference Range', 'Score']

In [86]:
def add_column_header(df, previous_column):
  col_list = ["Encounter", "Specimen", "Service Date", "High Score", "Score", "Lab", "Doctor Name", "Location", "Phone"]

  curr_rename_dict = {}
  curr_new_dict = {}

  curr_columns_values = list(df.columns)
  
  if previous_column == "Encounter":
    curr_columns_values.insert(1, "")

  for idx, col_name in enumerate(curr_columns_values):
    print(idx)
    if idx == 1:
      curr_rename_dict["Specimen"] = ""
    else:
      curr_rename_dict[col_name] = col_list[idx]

  for idx, col_name in enumerate(curr_columns_values):
    curr_new_dict[col_list[idx]] = col_name

  if previous_column == "Encounter":
    # renaming the columns
    df.rename(columns=curr_rename_dict, inplace = True)

  # adding a new row
  new_row_df = pd.DataFrame(curr_new_dict, index =[0])
  # Concatenate new_row with df
  updated_df = pd.concat([new_row_df, df[:]]).reset_index(drop = True)
  return updated_df

In [50]:
# drop unwanted columns
data_df = drop_unwanted_columns(dfs[13])
data_df.head()

,Test Name,Results,Reference Range,Score
0,HEMOGLOBIN,11.3 L,11.7-15.5 g/dL,5
1,RDW,16.5 H,11.0-15.0 %,5
2,Encounter,Specimen,Service Date,High Score
3,25,NaN,2021-06-23,5
4,NaN,NaN,NaN,NaN


In [ ]:
data_df2 = add_new_columns(data_df)
# Concatenate new_row with df
# data_df3 = pd.concat([data_df, data_df2[:]]).reset_index(drop = True)
data_df2.head()

In [ ]:
raw_df = data_df[1].rename(columns=data_df[1].iloc[0]).drop(data_df[1].index[0])
raw_df.columns

Index(['PLATELET COUNT', '299', '140-400 Thousand/uL', 0.0, nan, nan], dtype='object')

In [ ]:
for col in data_df.columns:
  print(col)

Test Name
Results
Reference Range
Score


In [ ]:
test_column_list = ["Test Name", "Results","Reference Range", "Score", "Lab"]
diagnosis_column_list = ["Diagnosis Code", "Description","Code Type", "Score"]
doctor_sp_column_list = ["Doctor Specialty", "Score"]
lab_column_list = ["Lab", "Name","Location", "Director", "CLIA"]
cols_list = ["Encounter", "Specimen", "Service Date", "High Score", "Score", "Lab", "Doctor Name", "Location", "Phone"]
#new_dict = {col_name in test_column_list}
current_dict = {}
for idx, col_name in enumerate(data_df.columns):
  print(f"{cols_list[idx]} >> {col_name}")
  current_dict[cols_list[idx]] = col_name

Encounter >> Test Name
Specimen >> Results
Service Date >> Reference Range
High Score >> Score


In [ ]:
current_dict

{'Encounter': 'Test Name',
 'Specimen': 'Results',
 'Service Date': 'Reference Range',
 'High Score': 'Score'}

In [ ]:
# renaming the columns
data_df.rename(columns={
  "Test Name": "Encounter",
  "Results": "Specimen",
  "Reference Range": "Service Date",
  "Score": "High Score"
}, inplace = True)
#data_df.head()

# adding a new row
new_row = pd.DataFrame({"Encounter":"Test Name", "Specimen":"Results", "Service Date":"Reference Range", "High Score":"Score"}, index =[0])
# Concatenate new_row with df
test_data_df = pd.concat([new_row, data_df[:]]).reset_index(drop = True)
test_data_df.head()

In [ ]:
data_df = []
for idx in range(len(dfs)):
  # filter unwanted tables
  if (dfs[idx].columns[0] != "Unnamed: 0") and (dfs[idx].columns[0] != "Condition") and (dfs[idx].columns[0] != "Response Status"):
    data_df.append(dfs[idx])

In [ ]:
dfs[11]

,Encounter,Specimen,Service Date,High Score,Unnamed: 0,Unnamed: 1
0,26.0,NaN,2021-08-17,5.0,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# drop unwanted columns
data_df = drop_unwanted_columns(dfs[11])

In [79]:
def create_data_df(dfs):
  data_df = pd.DataFrame()
  previous_column = ""
  for idx in range(len(dfs)):
    # Step-1: drop unwanted columns
    raw_df = drop_unwanted_columns(dfs[idx])

    # Step-1: skip loop if columns does not exists
    if len(raw_df.columns) == 0:
      continue

    # Step-2: if it is "Test Name", "Diagnosis Code", "Doctor Specialty" and "Lab" table             
    if raw_df.columns[0] not in ["Response Status", "Condition"]:
      # Step-3: if it is first "Encounter" table
      if len(data_df) != 9 and raw_df.columns[0] == "Encounter":
        cols_list = ["Score", "Lab", "Doctor Name", "Location", "Phone"]
        for col_name in cols_list:
          raw_df[col_name] = ""

      # Step-4: handle table that dont have column header
      if raw_df.columns[0] not in ["Encounter", "Test Name", "Diagnosis Code", "Doctor Specialty", "Lab", "21", "0"]:
        previous_column = raw_df.columns[0]
        add_column_header(raw_df, previous_column)

      # Step-5: otherwise, assign column value to "Encounter" table's column based on index position
      updated_df = update_df_columns(raw_df)  # rename and add new column to df
      data_df = pd.concat([data_df, updated_df], ignore_index=True)

  return data_df

In [ ]:
data_df = create_data_df(dfs)

In [82]:
len(data_df)

1221

In [83]:
data_df.head()

,Encounter,Specimen,Service Date,High Score,Score,Lab,Doctor Name,Location,Phone
0,Encounter,Specimen,Service Date,High Score,Score,Lab,Doctor Name,Location,Phone
1,30,NaN,2021-09-24,5,,,,,
2,NaN,NaN,NaN,NaN,,,,,
3,Diagnosis Code,Description,Code Type,Score,,,,,
4,D649,"Anemia, unspecified",ICD10,5,,,,,


In [84]:
data_df = data_df.drop([data_df.index[0]])
data_df.head()

,Encounter,Specimen,Service Date,High Score,Score,Lab,Doctor Name,Location,Phone
1,30,NaN,2021-09-24,5,,,,,
2,NaN,NaN,NaN,NaN,,,,,
3,Diagnosis Code,Description,Code Type,Score,,,,,
4,D649,"Anemia, unspecified",ICD10,5,,,,,
5,E611,Iron deficiency,ICD10,5,,,,,


In [85]:
data_df.to_csv("exam-one.csv")

In [ ]:
all_data_df = pd.concat(data_df)

In [ ]:
all_data_df.reset_index(drop=True, inplace=True)

##Raw Data

In [ ]:
all_data_df.columns

Index(['Unnamed: 0', 'Condition', 'Provider\rGenerated',
       'Demographic\rPrevalence', 'Confidence', 'Contributing Factors',
       'Response Status', 'Total # of Encounters', 'Encounter', 'Specimen',
       'Service Date', 'High Score', 'Unnamed: 1', 'Unnamed: 2', 'Test Name',
       'Results', 'Reference Range', 'Score', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', '21', '9', '2018-03-20', '5', 'Doctor Name',
       'Location', 'Phone', 'I951', 'Orthostatic hypotension', 'ICD10', 'Lab',
       '0', 'Doctor Specialty', 'N920',
       'Excessive and frequent menstruation with regular cycle',
       'Diagnosis Code', 'Description', 'Code Type', 'R000',
       'Tachycardia, unspecified', 'Name', 'Director', 'CLIA'],
      dtype='object')

In [ ]:
all_data_df.head()

,Unnamed: 0,Condition,Provider\rGenerated,Demographic\rPrevalence,Confidence,Contributing Factors,Response Status,Total # of Encounters,Encounter,Specimen,...,N920,Excessive and frequent menstruation with regular cycle,Diagnosis Code,Description,Code Type,R000,"Tachycardia, unspecified",Name,Director,CLIA
0,NaN,Oth symptoms and signs w\rcognitive functions ...,False,1.4%,99.0%,"Amorphous Sediment [Presence],Platelet\r[Prese...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Any Muscle Disease,True,43.5%,89.5%,"R29-Nervous Systom\rSymp.,Sodium,Hydromorphone...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Abnormalities of breathing,False,5.7%,57.2%,"LE,HDL,Free T4 Index,GFR - MDRD",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Other anemias,True,4.0%,52.2%,"Total Bilirubin,ALP,Hemoglobin,D50-Anemia(Iron)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,Hit,30.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# drop unwanted columns
drop_columns = [
  'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
  'Unnamed: 5', 'Unnamed: 6', '21', '9', '2018-03-20', '5',
  '0', 'Excessive and frequent menstruation with regular cycle',
  'Tachycardia, unspecified', 
]
data_df = all_data_df[all_data_df.columns.drop(drop_columns)]
data_df.shape

(1163, 31)

In [ ]:
data_df.columns

Index(['Condition', 'Provider\rGenerated', 'Demographic\rPrevalence',
       'Confidence', 'Contributing Factors', 'Response Status',
       'Total # of Encounters', 'Encounter', 'Specimen', 'Service Date',
       'High Score', 'Test Name', 'Results', 'Reference Range', 'Score',
       'Doctor Name', 'Location', 'Phone', 'I951', 'Orthostatic hypotension',
       'ICD10', 'Lab', 'Doctor Specialty', 'N920', 'Diagnosis Code',
       'Description', 'Code Type', 'R000', 'Name', 'Director', 'CLIA'],
      dtype='object')

In [ ]:
# Drop row that has all NaN values
data_df= data_df.dropna(how="all", axis=0)
data_df.shape

(1026, 31)

In [ ]:
# save to json/csv file
data_df.to_json('raw_data.json', orient="columns")
data_df.to_csv('raw_data.csv')

##Refined Data

In [ ]:
# drop unwanted columns
drop_columns = [
  'Unnamed: 0', 'Condition', 'Provider\rGenerated',
  'Demographic\rPrevalence', 'Confidence', 'Contributing Factors',
  'Response Status', 'Total # of Encounters', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
  'Unnamed: 5', 'Unnamed: 6', '21', '9', '2018-03-20', '5', 'Doctor Name',
  'Location', 'Phone', 'I951', 'Orthostatic hypotension', 'ICD10', 'Lab',
  '0', 'Doctor Specialty', 'N920',
  'Excessive and frequent menstruation with regular cycle',
  'Diagnosis Code', 'Description', 'Code Type', 'R000',
  'Tachycardia, unspecified', 'Name', 'Director', 'CLIA'
]
data_df = all_data_df[all_data_df.columns.drop(drop_columns)]
data_df.shape

(1163, 8)

In [ ]:
data_df.columns

Index(['Encounter', 'Specimen', 'Service Date', 'High Score', 'Test Name',
       'Results', 'Reference Range', 'Score'],
      dtype='object')

In [ ]:
# Drop row that has all NaN values
data_df= data_df.dropna(how="all", axis=0)
data_df.shape

(1007, 8)

In [ ]:
data_df.head()

,Encounter,Specimen,Service Date,High Score,Test Name,Results,Reference Range,Score
5,30,NaN,2021-09-24,5,NaN,NaN,NaN,NaN
7,Diagnosis Code,Description,Code Type,Score,NaN,NaN,NaN,NaN
8,D649,"Anemia, unspecified",ICD10,5,NaN,NaN,NaN,NaN
9,E611,Iron deficiency,ICD10,5,NaN,NaN,NaN,NaN
10,I951,Orthostatic hypotension,ICD10,5,NaN,NaN,NaN,NaN


In [ ]:
# split dataframe
encounter_df_columns = ["Encounter", "Specimen", "Service Date", "High Score"]
test_df_columns = ["Test Name",	"Results", "Reference Range",	"Score"]
test_data_df = data_df[test_df_columns]
encounter_data_df = data_df[encounter_df_columns]
(test_data_df.shape, encounter_data_df.shape)

((1007, 4), (1007, 4))

In [ ]:
# Drop row that has all NaN values
test_data_df= test_data_df.dropna(how="all", axis=0)
encounter_data_df= encounter_data_df.dropna(how="all", axis=0)
(test_data_df.shape, encounter_data_df.shape)

((783, 4), (224, 4))

In [ ]:
test_data_df.head()

,Test Name,Results,Reference Range,Score
23,HEMOGLOBIN,11.3 L,11.7-15.5 g/dL,5
24,RDW,16.5 H,11.0-15.0 %,5
25,Encounter,Specimen,Service Date,High Score
26,25,NaN,2021-06-23,5
28,Diagnosis Code,Description,Code Type,Score


In [ ]:
# renaming the columns
test_data_df.rename(columns={
  "Test Name": "Encounter",
  "Results": "Specimen",
  "Reference Range": "Service Date",
  "Score": "High Score"
}, inplace = True)
test_data_df.head()

,Encounter,Specimen,Service Date,High Score
23,HEMOGLOBIN,11.3 L,11.7-15.5 g/dL,5
24,RDW,16.5 H,11.0-15.0 %,5
25,Encounter,Specimen,Service Date,High Score
26,25,NaN,2021-06-23,5
28,Diagnosis Code,Description,Code Type,Score


In [ ]:
# adding a new row
new_row = pd.DataFrame({"Encounter":"Test Name", "Specimen":"Results", "Service Date":"Reference Range", "High Score":"Score"}, index =[0])
# Concatenate new_row with df
test_data_df = pd.concat([new_row, test_data_df[:]]).reset_index(drop = True)
test_data_df.head()

,Encounter,Specimen,Service Date,High Score
0,Test Name,Results,Reference Range,Score
1,HEMOGLOBIN,11.3 L,11.7-15.5 g/dL,5
2,RDW,16.5 H,11.0-15.0 %,5
3,Encounter,Specimen,Service Date,High Score
4,25,NaN,2021-06-23,5


In [ ]:
single_data_df = pd.concat([encounter_data_df, test_data_df])
single_data_df.reset_index(drop=True, inplace=True)
single_data_df.shape

(1008, 4)

In [ ]:
single_data_df.head()

,Encounter,Specimen,Service Date,High Score
0,30,NaN,2021-09-24,5
1,Diagnosis Code,Description,Code Type,Score
2,D649,"Anemia, unspecified",ICD10,5
3,E611,Iron deficiency,ICD10,5
4,I951,Orthostatic hypotension,ICD10,5


In [ ]:
# save to json/csv file
single_data_df.to_json('refined_data.json', orient="columns")
single_data_df.to_csv('refined_data.csv')

##Clean Data

In [ ]:
# Drop all rows with column 'Test Name' and Reference Range has value 'Test Name', 'Diagnosis Code' and 'ICD10'
index_encounter_value = encounter_data_df[(encounter_data_df['Encounter'] == 'Diagnosis Code') | (encounter_data_df['Service Date'] == 'ICD10') | 
                                          (encounter_data_df['Service Date'] == 'ICD9')].index
index_test_value = test_data_df[(test_data_df['Test Name'] == 'Diagnosis Code') | 
                                (test_data_df['Reference Range'] == 'ICD10')].index
encounter_data_df = encounter_data_df.drop(index_encounter_value)
test_data_df = test_data_df.drop(index_test_value)
(test_data_df.shape, encounter_data_df.shape)

((740, 4), (139, 4))

In [ ]:
encounter_data_df.head(20)

,Encounter,Specimen,Service Date,High Score
0,30,NaN,2021-09-24,5
9,Test Name,Results,Reference Range,Score
10,GLUCOSE,100 H,65-99 mg/dL,5
11,WHITE BLOOD CELL COUNT,3.2 L,3.8-10.8 Thousand/uL,5
12,RDW,15.6 H,11.0-15.0 %,5
13,ABSOLUTE NEUTROPHILS,1459 L,1500-7800 cells/uL,5
14,26.0,NaN,2021-08-17,5.0
138,10.0,NaN,2019-02-19,10.0
179,2,NaN,2016-10-06,10
181,Doctor Specialty,Score,NaN,NaN


In [ ]:
# save to json file
encounter_data_df.to_json('test1.json', orient="columns")

In [ ]:
!cat test1.json

In [ ]:
test_data_df.head(20)

,Test Name,Results,Reference Range,Score
16,HEMOGLOBIN,11.3 L,11.7-15.5 g/dL,5
17,RDW,16.5 H,11.0-15.0 %,5
18,Encounter,Specimen,Service Date,High Score
19,25,NaN,2021-06-23,5
25,Test Name,Results,Reference Range,Score
26,"IRON, TOTAL",31 L,40-190 mcg/dL,5
27,% SATURATION,8 L,16-45 % (calc),5
28,FERRITIN,3 L,16-154 ng/mL,5
29,RED BLOOD CELL COUNT,3.75 L,3.80-5.10 Million/uL,5
30,HEMOGLOBIN,10.3 L,11.7-15.5 g/dL,5


In [ ]:
# save to json file
test_data_df.to_json('test2.json', orient="columns")

In [ ]:
!cat test2.json

In [ ]:
data_df["Results"].astype(str) == "nan"

0     False
1     False
2     False
3      True
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18     True
24    False
25    False
26    False
27    False
28    False
29     True
Name: Results, dtype: bool

In [ ]:
# splitting dataframe by groups
grouped = data_df.groupby(data_df["Results"].astype(str))
service_date_df = grouped.get_group("nan")
service_date_df

,Test Name,Results,Reference Range,Score
3,25,NaN,2021-06-23,5
18,24,NaN,2021-06-03,5
29,23,NaN,2021-04-29,5


In [ ]:
# save to json file
out_file = data_df.to_json('test.json', date_format='iso', orient='records')

In [ ]:
# save to json file
out_file = data_df.to_json('test.json', orient="columns")

In [ ]:
!cat test.json

In [ ]:
def save_data_into_json(df, idx):
  # drop column that has named "Unnamed: xxx"
  data_df = df[df.columns.drop(list(df.filter(regex="Unnamed")))]
  # Drop row that has all NaN values
  data_df= data_df.dropna(how="all", axis=0)
  # save to json file
  data_df.to_json(f"data/test-{idx}.json", date_format="iso", orient="records")

In [ ]:
!mkdir data

In [ ]:
for idx in range(len(dfs)):
  if dfs[idx].columns[0] != "Unnamed: 0":
    save_data_into_json(dfs[idx], idx)

##Further practice

In [ ]:
tabula.read_pdf("redacted1.pdf", pages="all", stream=True)

In [ ]:
dfs = tabula.read_pdf(
    "redacted1.pdf",
    lattice=True,
    #pandas_options={"header": [0, 1]},
    # area=[0, 0, 50, 100],
    relative_area=True,
    multiple_tables=True,
    pages=1,
)

Oct 18, 2022 10:37:59 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
df = dfs[0].drop(["Unnamed: 0"], axis=1)
df

""


In [ ]:
len(df)

0

In [ ]:
dfs[9]

,Unnamed: 0,Unnamed: 1
0,NaN,NaN


In [ ]:
len(dfs[6].value_counts())

4

In [ ]:
data_idx = []
for idx in range(len(dfs)):
  if len(dfs[idx].value_counts()) > 0:
    data_idx.append(idx)

In [ ]:
data_idx

[6, 7]

In [ ]:
dfs[8]

,Encounter,Specimen,Service Date,High Score,Unnamed: 0,Unnamed: 1
0,30,NaN,2021-09-24,5,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Diagnosis Code,Description,Code Type,Score,NaN,NaN
3,D649,"Anemia, unspecified",ICD10,5,NaN,NaN
4,E611,Iron deficiency,ICD10,5,NaN,NaN
5,I951,Orthostatic hypotension,ICD10,5,NaN,NaN
6,N920,Excessive and frequent menstruation with regul...,ICD10,5,NaN,NaN
7,R197,"Diarrhea, unspecified",ICD10,5,NaN,NaN
8,R7889,"Finding of other specified substances, not nor...",ICD10,5,NaN,NaN
9,Test Name,Results,Reference Range,Score,NaN,NaN


In [ ]:
data_df = None
for idx in range(len(dfs)):
  if len(dfs[idx].value_counts()) > 0:
    data_df = dfs[idx]
data_df

,Condition,Provider\rGenerated,Demographic\rPrevalence,Confidence,Contributing Factors
0,Oth symptoms and signs w\rcognitive functions ...,False,1.4%,99.0%,"Amorphous Sediment [Presence],Platelet\r[Prese..."
1,Any Muscle Disease,True,43.5%,89.5%,"R29-Nervous Systom\rSymp.,Sodium,Hydromorphone..."
2,Abnormalities of breathing,False,5.7%,57.2%,"LE,HDL,Free T4 Index,GFR - MDRD"
3,Other anemias,True,4.0%,52.2%,"Total Bilirubin,ALP,Hemoglobin,D50-Anemia(Iron)"


In [ ]:
tabula.convert_into("redacted1.pdf", "test.json", output_format="json")

Oct 18, 2022 9:53:01 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
!cat test.json

[{"extraction_method":"stream","top":249.0,"left":41.0,"width":523.0,"height":116.0,"right":564.0,"bottom":365.0,"data":[[{"top":256.81,"left":71.71,"width":48.05000305175781,"height":4.630000114440918,"text":"Condition"},{"top":256.81,"left":161.93,"width":43.350006103515625,"height":4.630000114440918,"text":"Provider"},{"top":256.81,"left":222.26,"width":66.68000793457031,"height":4.630000114440918,"text":"Demographic"},{"top":256.81,"left":300.38,"width":55.839996337890625,"height":4.630000114440918,"text":"Confidence"},{"top":256.81,"left":414.65,"width":98.8900146484375,"height":4.630000114440918,"text":"Contributing Factors"}],[{"top":0.0,"left":0.0,"width":0.0,"height":0.0,"text":""},{"top":267.97,"left":158.9,"width":50.850006103515625,"height":4.630000114440918,"text":"Generated"},{"top":267.97,"left":229.18,"width":52.810001373291016,"height":4.630000114440918,"text":"Prevalence"},{"top":0.0,"left":0.0,"width":0.0,"height":0.0,"text":""},{"top":0.0,"left":0.0,"width":0.0,"hei

In [ ]:
tabula.convert_into("redacted1.pdf", "test.tsv", output_format="tsv", 
                    pages='all', lattice=True, area=[0, 0, 50, 100],
                    relative_area=True,
                    )

Oct 18, 2022 9:58:42 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
!cat test.tsv

In [ ]:
import pandas as pd
import json

In [ ]:
master_df = pd.read_csv('test.tsv', sep='\s+', encoding='utf-8', on_bad_lines='skip')

In [ ]:
out_file = master_df.to_json('test.json', date_format='iso', orient='records')

In [ ]:
!cat test.json